In [ ]:
import matplotlib.pyplot as plt
import numpy as np

import sys
sys.path.append('..')

In [10]:
import os

OUTPUT_DIR = '/home/azureuser/cloudfiles/code/Users/e1310988/satellite-te/output'
PROBLEM_PREFIX = 'DataSetForSaTE'

INTENSITY_LIST = ['25', '50', '75', '100']
# TEAL_OUTPUT_LIST = [os.path.join(OUTPUT_DIR, f'{PROBLEM_PREFIX}_Int{intensity}_teal') for intensity in INTENSITY_LIST]
# LP_OUTPUT_LIST = [os.path.join(OUTPUT_DIR, f'{PROBLEM_PREFIX}_Int{intensity}_lp') for intensity in INTENSITY_LIST]
SATE_OUTPUT_LIST_ISL = [os.path.join(OUTPUT_DIR, f'{PROBLEM_PREFIX}{intensity}_ISL_spaceTE') for intensity in INTENSITY_LIST]

In [11]:
import pandas as pd

# TEAL_TEST_LOG_FNAME = 'teal_topo-300_tsz-None_vr-0.2_lr-0.0001_ep-3_bsz-20_layer-6_sample-5_rho-1.0_step-5.csv'
# LP_TEST_LOG_FNAME = 'gurobi_topo-300_tsz-None.csv'
SATE_TEST_LOG_FNAME = 'spaceTE_topo-10000_tsz-None_vr-0.2_lr-0.0001_ep-3_bsz-32_sample-5_rho-1.0_step-5_quantized-False_compiled-False.csv'

# teal_df_list = [pd.read_csv(os.path.join(AssetManager.test_log_dir(output), TEAL_TEST_LOG_FNAME)) \
#     for output in TEAL_OUTPUT_LIST]
# lp_df_list = [pd.read_csv(os.path.join(AssetManager.test_log_dir(output), LP_TEST_LOG_FNAME)) \
#     for output in LP_OUTPUT_LIST]"
sate_test_list = [pd.read_csv(os.path.join(output, "test_logs", SATE_TEST_LOG_FNAME)) \
    for output in SATE_OUTPUT_LIST_ISL]

algo_df_dict = {
    'SATE': sate_test_list
}

In [12]:
sate_mean = [df.mean() for df in sate_test_list]

In [13]:
sate_mean

[topo_idx          999.500000
 tm_idx            999.500000
 total_demand    36877.220137
 obj_val         29486.858318
 ratio               0.799844
 runtime             0.017913
 dtype: float64,
 topo_idx          999.500000
 tm_idx            999.500000
 total_demand    74096.772410
 obj_val         45360.538691
 ratio               0.612350
 runtime             0.017674
 dtype: float64,
 topo_idx           999.500000
 tm_idx             999.500000
 total_demand    111265.565809
 obj_val          58255.861008
 ratio                0.523657
 runtime              0.017964
 dtype: float64,
 topo_idx           999.500000
 tm_idx             999.500000
 total_demand    147316.266867
 obj_val          67069.945867
 ratio                0.455362
 runtime              0.017223
 dtype: float64]

Bad pipe message: %s [b'\xbbR\xf5\x9bWh\xd0\xb46\x7f\xdf N\xb2W\x08%\x04 S\xa1\xb8\xbc\xed\x96\x807\xc7\r\xf0\xf1)X\xd6\x08\r\xb8\xbbh$\x10E:\x97\x94U\x1a\xdd\xb1\x9f\x16\x00\x08\x13\x02\x13\x03\x13\x01\x00\xff\x01\x00\x00\x8f\x00\x00\x00\x0e\x00\x0c\x00\x00\t127.0.0.1\x00\x0b\x00\x04\x03\x00\x01\x02\x00\n\x00\x0c\x00\n\x00\x1d\x00\x17\x00\x1e\x00\x19\x00\x18\x00#\x00\x00\x00\x16\x00\x00\x00\x17\x00\x00\x00\r\x00\x1e\x00\x1c\x04\x03\x05\x03\x06\x03\x08\x07\x08\x08\x08\t\x08\n\x08\x0b\x08\x04\x08\x05\x08\x06\x04\x01\x05\x01\x06\x01\x00+\x00\x03\x02\x03', b'-\x00\x02']
Bad pipe message: %s [b'0\xd2hUr\x95R(\xf5\x94\x1c\xc2\xd1\x1b\xb76E\xe1\x00\x00|\xc0,\xc00\x00\xa3\x00\x9f\xcc\xa9\xcc\xa8\xcc\xaa\xc0\xaf\xc0\xad\xc0\xa3\xc0\x9f\xc0]\xc0a\xc0W\xc0S\xc0+\xc0/\x00\xa2\x00\x9e\xc0\xae\xc0\xac\xc0\xa2\xc0\x9e\xc0\\\xc0`\xc0V\xc0R\xc0$\xc0(\x00']
Bad pipe message: %s [b"j\xc0#\xc0'\x00g\x00@\xc0\n\xc0\x14\x009\x008\xc0\t\xc0\x13\x003\x002\x00\x9d\xc0\xa1\xc0\x9d\xc0Q\x00\x9c\xc0\xa0\xc0\x9c\

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Wed Apr 10 09:57:26 2024

@author: WHH
"""
import numpy as np
import MultiShellGraph as MSG
import pickle
import time
import copy
import networkx as nx
import scipy.io as sio
import SPOnGrid as SPG
import ECMP as Baseline1
# ========= Parameters to change ==========
DataSetSize = 100 #
fileName = open("C:/Users/admin/Desktop/Learning/论文_NSDI/StarLink/DataSetForSaTE/DataSetForSaTE50/StarLink_DataSetForAgent50_5000_A.pkl", "rb")
InterConnectedMode = 'ISL'#'GrdStation' # 'ISL'
ISLCap = 50*4
UpLinkCap = 200*4
DownLinkCap = 200*4

# ========= Orbit Shell Parameters =========
OrbitNum1 = 72
SatNum1   = 22

OrbitNum2 = 72
SatNum2   = 22

OrbitNum3 = 58
SatNum3   = 6

OrbitNum4 = 36
SatNum4   = 20

GrdStationNum = 222

# =========== Generate Intra-Shell Static Graph =========
LatMat1 = [0 for i in range(OrbitNum1 * SatNum1)]
LatMat2 = [0 for i in range(OrbitNum2 * SatNum2)]
LatMat3 = [0 for i in range(OrbitNum3 * SatNum3)]
LatMat4 = [0 for i in range(OrbitNum4 * SatNum4)]
LatLimit = 90

Offset1 = 0
Offset2 = OrbitNum1 * SatNum1
Offset3 = OrbitNum1 * SatNum1 + OrbitNum2 * SatNum2
Offset4 = OrbitNum1 * SatNum1 + OrbitNum2 * SatNum2 + OrbitNum3 * SatNum3
Offset5 = OrbitNum1 * SatNum1 + OrbitNum2 * SatNum2 + OrbitNum3 * SatNum3 + OrbitNum4 * SatNum4

G1, EMap1, E1 = MSG.Inter_Shell_Graph(OrbitNum1, SatNum1, LatMat1, Offset1, LatLimit)
G2, EMap2, E2 = MSG.Inter_Shell_Graph(OrbitNum2, SatNum2, LatMat2, Offset2, LatLimit)
G3, EMap3, E3 = MSG.Inter_Shell_Graph(OrbitNum3, SatNum3, LatMat3, Offset3, LatLimit)
G4, EMap4, E4 = MSG.Inter_Shell_Graph(OrbitNum4, SatNum4, LatMat4, Offset4, LatLimit)

G_Trajectory = []
ISL_Trajectory = []
G_interShell_last = []
ISL_interShell_last = []
Result = [[],[],[]]
# =========== Load DataSet ==============
for k in range(DataSetSize):
    data = pickle.load(fileName)
    G_interShell = data['InterShell_GrdRelay']
    ISL_interShell = data['InterShell_ISL']
    FlowSet = data['FlowSet']
    print(['Index of Loaded Data:', k])
    
    # =================================================
    # --------------- Generate E and G ----------------
    # =================================================
    if InterConnectedMode == 'GrdStation':
        # Generate E
        E_inter = []
        for SatIndex in range(len(G_interShell)):
            if int(G_interShell[SatIndex]) >= 0:
                E_inter.append([SatIndex, int(G_interShell[SatIndex] + Offset5)])
                E_inter.append([int(G_interShell[SatIndex] + Offset5), SatIndex])    
        # G_interShell = np.zeros(4236) - 1
        
        if not np.array_equal(G_interShell,G_interShell_last):
            G_Trajectory.append(k)
            G_interShell_last = copy.deepcopy(G_interShell)
    else:
        # Generate E
        E_inter = []
        for SatIndex in range(len(ISL_interShell[0])): # 2 to 1
            if ISL_interShell[0][SatIndex] >= 0:
                S2 = SatIndex + Offset2 
                S1 = int(ISL_interShell[0][SatIndex])
                E_inter.append([S2, S1])
                E_inter.append([S1, S2])
        
        for SatIndex in range(len(ISL_interShell[1])): # 3 to 2
            if ISL_interShell[1][SatIndex] >= 0:
                S3 = int(SatIndex + Offset3)
                S2 = int(ISL_interShell[1][SatIndex] + Offset2)
                E_inter.append([S3, S2])
                E_inter.append([S2, S3])
        
        for SatIndex in range(len(ISL_interShell[2])): # 4 to 3
            if ISL_interShell[2][SatIndex] >= 0:
                S4 = int(SatIndex + Offset4)
                S3 = int(ISL_interShell[2][SatIndex] + Offset3)
                E_inter.append([S4, S3])
                E_inter.append([S3, S4])
        
        if ISL_interShell != ISL_interShell_last:
            ISL_Trajectory.append(k)
            ISL_interShell_last = copy.deepcopy(ISL_interShell)
    
    E = E1 + E2 + E3 + E4 + E_inter
    # print(['Number of Edge:', len(E)])
    # Generate G
    G = np.zeros((Offset5 + GrdStationNum, Offset5 + GrdStationNum)) + 999
    for Edge in E:
        G[Edge[0]][Edge[1]] = 1
    # =================================================
    # sio.savemat('TrackChanges.mat',{'G_Trajectory':G_Trajectory,
    #                                 'ISL_Trajectory':ISL_Trajectory})
    # ======================== Routing ================
    
    # E_load = [0 for x in range(len(E))]
    # for flow in FlowSet :
    #     Path = SPG.SPOnGrid(flow[0],flow[1],
    #                     G_interShell, 
    #                     ISL_interShell, 
    #                     InterConnectedMode, 
    #                     5) 
    #     for p in Path:
    #         for node in range(len(p)-1):
    #             x = E.index([int(p[node]),int(p[node+1])])
    #             E_load[x] = E_load[x] + flow[2]/5
    # print(sum(min(200,x) for x in E_load) / sum(x > 0 for x in E_load))
    # print(sum(x for x in E_load) / sum(x > 0 for x in E_load))   
        # for p in Path:
        #     if p[0] != flow[0] or p[-1] != flow[1]:
        #         print('Src Des Error!')
        #         break
        #     p_set = set(p)
        #     if len(p_set) != len(p):
        #         print('Loop!')
        #         break
        #     for node in range(len(p)-1):
        #         if G[int(p[node])][int(p[node+1])] != 1:
        #             print(['Invalid Edge!',int(p[node]), int(p[node+1])])
        #             break
    # for grd in range(222):
    #     print(SPG.SatOverGrdStation(grd,G_interShell))
    # ============== ECMP ====================
    time_M0 = time.perf_counter()
    Throughput, Ratio = Baseline1.ECMP(E,FlowSet,G_interShell,ISL_interShell,InterConnectedMode,ISLCap,UpLinkCap,DownLinkCap)
    time_M1 = time.perf_counter()
    print([Throughput, Ratio, time_M1- time_M0])
    Result[0].append(Throughput)
    Result[1].append(Ratio)
    Result[2].append(time_M1- time_M0)

print([sum(Result[0])/len(Result[0]),
       sum(Result[1])/len(Result[1]),
       sum(Result[2])/len(Result[2])])
            
    # ================ Isomorphism ==================
    # if k == 0:
    #     Purified_Topology = []  
    # IsANewTopology = 1
    # for Paras in Purified_Topology:  
    #     start = time.perf_counter()
    #     TT = nx.Graph()
    #     TT.add_nodes_from([i for i in range(Offset5 + GrdStationNum)])
    #     TT.add_edges_from(E)
    #     T0 = nx.Graph()
    #     T0.add_nodes_from([i for i in range(Offset5 + GrdStationNum)])
    #     T0.add_edges_from(E1+E2+E3+E4+Paras)
    #     GM = nx.isomorphism.GraphMatcher(T0, TT)
    #     # print(GM.is_isomorphic())
    #     if nx.is_isomorphic(T0, TT) is True:            
    #         # mapping = GM.mapping
    #         IsANewTopology = 0
    #         break
    #     end = time.perf_counter() 
    #     print(end-start)
    # if IsANewTopology == 1:
    #     Purified_Topology.append(E_inter)
    # print([len(G_Trajectory),len(ISL_Trajectory),len(Purified_Topology)])   
    